In [1]:
import os

In [2]:
pwd

'/Users/salonisahal/Desktop/Text-Summarizer-/research'

In [3]:
os.chdir("../")

In [4]:
pwd

'/Users/salonisahal/Desktop/Text-Summarizer-'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directory

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH):
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments

          create_directory([config.root_dir])

          model_trainer_config= ModelTrainerConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                model_ckpt=config.model_ckpt,
                num_train_epochs=params.num_train_epochs,
                warmup_steps=params.warmup_steps,
                per_device_train_batch_size= params.per_device_train_batch_size,
                weight_decay=params.weight_decay,
                logging_steps=params.logging_steps,
                evaluation_strategy= params.evaluation_strategy,
                eval_steps= params.eval_steps,
                save_steps=params.save_steps,
                gradient_accumulation_steps=params.gradient_accumulation_steps
          )
          return model_trainer_config

In [8]:
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch

/opt/anaconda3/envs/chicken/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-25 16:10:20,947: INFO: utils:NumExpr defaulting to 8 threads.]
[2025-04-25 16:10:21,690: INFO: config:PyTorch version 2.4.1 available.]
[2025-04-25 16:10:21,691: INFO: config:TensorFlow version 2.13.0 available.]


In [11]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config=config

    def train(self):
        torch.cuda.empty_cache()
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus= AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        # loading data
        dataset_samsum_pt=load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
        output_dir = 'pegasus-samsum',num_train_epochs=1,warmup_steps=500,
        per_device_train_batch_size=1,per_device_eval_batch_size=1,
        weight_decay=0.01,logging_steps=10,
        evaluation_strategy='steps',eval_steps=500,save_steps=1e6,
        gradient_accumulation_steps= 4
       
        )

        trainer=Trainer(model=model_pegasus,args=trainer_args,
                        tokenizer=tokenizer,data_collator=seq2seq_data_collator,
                        train_dataset=dataset_samsum_pt["test"],
                        eval_dataset=dataset_samsum_pt['validation'])
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        




        

In [12]:
try:
    config =ConfigurationManager()
    model_trainer_config =config.get_model_trainer_config()
    model_trainer_config= ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-25 16:32:37,630: INFO: common:yaml file:config/config.yaml loaded Successfully]
[2025-04-25 16:32:37,638: INFO: common:yaml file:params.yaml loaded Successfully]
[2025-04-25 16:32:37,640: INFO: common:Directory artifacts created successfully.]
[2025-04-25 16:32:37,643: INFO: common:Directory artifacts/model_trainer created successfully.]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/gn/0625w7jn7mscm9f5g6pchy800000gn/T/ipykernel_7809/4247446282.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model_pegasus,args=trainer_args,


RuntimeError: MPS backend out of memory (MPS allocated: 6.20 GB, other allocations: 2.57 GB, max allowed: 9.07 GB). Tried to allocate 375.40 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).